In [ ]:
!pip install yfinance pandas numpy scikit-learn plotly apscheduler

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings

In [ ]:
def get_stock_data(ticker, period="1y"):
    stock = yf.Ticker(ticker)
    df = stock.history(period=period)
    return df

def calculate_technical_indicators(df):
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()

    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    exp1 = df['Close'].ewm(span=12, adjust=False).mean()
    exp2 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()



In [ ]:
class StockPredictor:
    def __init__(self):
        self.model = RandomForestRegressor(n_estimators=100, random_state=42)

    def prepare_data(self, df):
        df = df.copy()

        df['Returns'] = df['Close'].pct_change()
        df['Volatility'] = df['Returns'].rolling(window=20).std()
        df['Price_Range'] = df['High'] - df['Low']
        df['Volume_Change'] = df['Volume'].pct_change()

        df['Target'] = df['Returns'].shift(-1)
        df = df.dropna()

        feature_columns = ['Returns', 'Volatility', 'Price_Range', 'Volume_Change',
                          'RSI', 'MACD']

        return df[feature_columns], df['Target']

    def train(self, df):
        X, y = self.prepare_data(df)
        self.model.fit(X, y)

    def predict(self, df):
        X, _ = self.prepare_data(df)
        return
        return self.model.predict(X)


In [ ]:
def analyze_stock(ticker):
    df = get_stock_data(ticker)
    df = calculate_technical_indicators(df)

    predictor = StockPredictor()
    predictor.train(df)

    predictions = predictor.predict(df)
    df['Predicted_Returns'] = predictions

    current_price = df['Close'].iloc[-1]
    predicted_return = predictions[-1]
    rsi = df['RSI'].iloc[-1]
    macd = df['MACD'].iloc[-1]
    signal_line = df['Signal_Line'].iloc[-1]

    analysis = {
        'ticker': ticker,
        'current_price': current_price,
        'predicted_return': predicted_return,
        'rsi': rsi,
        'recommendation': 'AANHOUDEN'
    }

    if predicted_return > 0.02 and rsi < 70 and macd > signal_line:
        analysis['recommendation'] = 'KOPEN'
    elif predicted_return < -0.02 or rsi > 70 or macd < signal_line:
        analysis['recommendation'] = 'VERKOPEN'

    return analysis, df

In [ ]:
def create_stock_dashboard(ticker, df):
    fig = make_subplots(rows=3, cols=1,
                        subplot_titles=('Prijs en Voortschrijdende Gemiddelden', 'RSI', 'MACD'),
                        vertical_spacing=0.1,
                        row_heights=[0.5, 0.25, 0.25])

    # Price and Moving Averages
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['Open'],
                                 high=df['High'],
                                 low=df['Low'],
                                 close=df['Close'],
                                 name='OHLC'),
                  row=1, col=1)

    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_20'],
                             name='SMA 20', line=dict(color='orange')),
                  row=1, col=1)

    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_50'],
                             name='SMA 50', line=dict(color='blue')),
                  row=1, col=1)

    # RSI
    fig.add_trace(go.Scatter(x=df.index, y=df['RSI'],
                             name='RSI', line=dict(color='purple')),
                  row=2, col=1)

    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

    # MACD
    fig.add_trace(go.Scatter(x=df.index, y=df['MACD'],
                             name='MACD', line=dict(color='blue')),
                  row=3, col=1)

    fig.add_trace(go.Scatter(x=df.index, y=df['Signal_Line'],
                             name='Signal Line', line=dict(color='orange')),
                  row=3, col=1)

    fig.update_layout(
        title=f'{ticker} Technische Analyse',
        xaxis_title='Datum',
        yaxis_title='Prijs',
        height=900,
        showlegend=True
    )

    return fig


In [ ]:

```
